<a href="https://colab.research.google.com/github/bchuchucaa/RNNConsumoDeAgua/blob/main/RNNConsumoDeAgua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/archive.zip

unzip:  cannot find or open /content/archive.zip, /content/archive.zip.zip or /content/archive.zip.ZIP.


In [1]:
from pandas import read_csv
from datetime import datetime
import pandas as pd
from matplotlib import pyplot
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

print('LIBRERIAS IMPORTADAS')

LIBRERIAS IMPORTADAS


**LECTURA DE DATOS**

In [2]:
#LOADING DATA 
df= read_csv('/content/AguaH.csv')
df = df.iloc[0:5000 , :]
#AJUSTE DEL FORMATO DE LAS FECHAS PARA LA TRANSFORMACION A TIPO DATETIME
lookup = {'ENE':'01','FEB':'02','MAR':'03','ABR':'04','MAY':'05','JUN':'06','JUL':'07','AGO':'08','SEP':'09','OCT':'10','NOV':'11','DIC':'12'}
clist=[]
for col in df.columns[5:]:
    col = col[4:].split('_')
    clist.append('20'+col[1]+'-'+lookup[col[0]]+'-01')

columns = ['LANDUSE_TYPE','USER','PIPE DIAM','VENDOR','JAN16']+clist
df.columns=columns
df = df.drop(columns=['LANDUSE_TYPE','PIPE DIAM','VENDOR','JAN16'])

#df=df.transpose()
print("Dataframe shape ",df.shape)


Dataframe shape  (5000, 85)


In [ ]:
#ESTE CODIGO SIRVE UNICAMENTE PARA LA INTEGRACION DE LOS NOMBRES EN EL DATAFRAME
col_list = ["Name"]
names = pd.read_csv("/content/StateNames.csv", usecols=col_list)
names = names.iloc[0:84 , :]
#ELIMINAMOS LA COLUMNA DE LOS NOMBRE YA QUE SE BINARIZARON
#df = df.iloc[: , 1:]
print("Names shape ",names.shape)

Names shape  (84, 1)


In [3]:
rows_list=[]
dataComp = pd.DataFrame(columns=('DATE','TIPO', 'CONSUMO'))
for column in range(1,len(df.columns),1):
  selection=df.iloc[:,column]
  for row in range(len(selection)):
    df2=[df.columns[column],df.iloc[row,0],selection.iloc[row]]
    rows_list.append(df2)
    
dataComp = pd.DataFrame(rows_list)  

In [6]:
dataComp.rename(columns = {0:'DATE',1:'TIPO',2:'CONSUMO'}, inplace = True)
dataComp['DATE'] = pd.to_datetime(dataComp['DATE'], format='%Y-%m-%d')
dataComp.set_index('DATE')

,TIPO,CONSUMO
DATE,,
2009-01-01,DOMESTICO MEDIO,20.0
2009-01-01,DOMESTICO MEDIO,NaN
2009-01-01,DOMESTICO RESIDENCIAL,NaN
2009-01-01,DOMESTICO MEDIO,NaN
2009-01-01,DOMESTICO RESIDENCIAL,20.0
...,...,...
2015-12-01,DOMESTICO MEDIO,22.0
2015-12-01,DOMESTICO MEDIO,3.0
2015-12-01,DOMESTICO MEDIO,8.0


In [7]:

#RELLENANDO VALORES NAN POR LA MEDIA
dataComp=dataComp.fillna(20)
#CONVIRTIENDO EL INDICE DEL DATAFRAME EN TIPO DATETIME
#dataComp.index = pd.to_datetime(dataComp.index)
#type(dataComp.index)

In [8]:
dataComp['DATE'].describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                  420000
unique                     84
top       2012-05-01 00:00:00
freq                     5000
first     2009-01-01 00:00:00
last      2015-12-01 00:00:00
Name: DATE, dtype: object

In [9]:
one_hot = pd.get_dummies(dataComp['TIPO'])
dataComp = dataComp.drop('TIPO',axis = 1)
dataComp = dataComp.join(one_hot)


In [10]:
values = dataComp.iloc[:,1:].values
train, test=train_test_split(values,test_size=.2,shuffle=False)
# asegurarse que todo esta en formato float
values = values.astype('float32')
# normalzación
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled

array([[1.9939782e-04, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.9939782e-04, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.9939782e-04, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [7.9759127e-05, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.1963869e-04, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.1933760e-04, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [19]:

numregistros=scaled.shape[0]
print(numregistros)

# split en conjuntos de train y test
n_train = int(round(numregistros*0.8,0)) #80% para entrenamiento
train = values[:n_train, :]
test = values[n_train:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

420000
(336000, 1, 7) (336000,) (84000, 1, 7) (84000,)


In [ ]:
# Diseño de la red recurrente LSTM
model = Sequential()
model.add(LSTM(24, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(128))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=100, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

Epoch 1/100
4667/4667 - 15s - loss: 0.0382 - val_loss: 0.0046 - 15s/epoch - 3ms/step
Epoch 2/100
4667/4667 - 13s - loss: 0.0081 - val_loss: 0.0029 - 13s/epoch - 3ms/step
Epoch 3/100
4667/4667 - 12s - loss: 0.0068 - val_loss: 0.0141 - 12s/epoch - 3ms/step
Epoch 4/100
4667/4667 - 13s - loss: 0.0057 - val_loss: 0.0027 - 13s/epoch - 3ms/step
Epoch 5/100


In [30]:
train_acc = model.evaluate(train_X, train_y, verbose=0)
test_acc = model.evaluate(test_X, test_y, verbose=0)

In [31]:
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))


Train: 0.005, Test: 0.005


In [ ]:
#FUNCION PARA ESCOGER UN REGISTRO DENTRO DEL DATAFRAME PARA LAS PRUEBAS DE VALIDACION
def selectRabbit(user):
  xnew = df.iloc[user:user+1 , 0:999]
  return xnew


In [ ]:
#FUNCION PARA BUSCAR EL VALOR REAL DE CONSUMO DENTRO DEL DATAFRAME
def searchForReal(user):
  real=xnew = df.iloc[user ,-1]
  return real

In [ ]:
#FUNCION PARA GRAFICAR LOS RESULTADOS
def graphResults(2):
  row = df.iloc[user:user+1 , 0:999]

In [ ]:
def prediccion(user):
    xnew=selectRabbit(user)
    Xnew= scaler.transform(xnew)
    Xnew = Xnew.reshape((Xnew.shape[0], 1, Xnew.shape[1]))
    pred= model.predict(Xnew)[0]
    return pred
    
pred=prediccion(83)
print('PREDICCION DE CONSUMO: ',pred)
print('USUARIO CONSUMO REAL :  [',searchForReal(83),']')

PRUEBAS PARA LA BINARIZACION DE LOS NOMBRES

In [ ]:
#TRYING TO TRANSPOSE DATA CONSUMPTION WITH NAMES OF CLIENTS
data= pd.read_csv('/content/AguaH.csv')
dataset= data.iloc[0:10,:]
dataset.shape
#ESTE CODIGO SIRVE UNICAMENTE PARA LA INTEGRACION DE LOS NOMBRES EN EL DATAFRAME
col_list = ["Name"]
names = pd.read_csv("/content/StateNames.csv", usecols=col_list)
names = names.iloc[0:10 , :]
#ELIMINAMOS LA COLUMNA DE LOS NOMBRE YA QUE SE BINARIZARON
#df = df.iloc[: , 1:]
print("Names shape ",names.shape)

In [ ]:
names.index = dataset.index
dataset.insert(loc=1, column='Names', value=names)


In [ ]:
dataset=dataset.transpose()



In [ ]:
dataset.index.name = 'date'
dataset


In [ ]:
dataset.set_index(['date', 'Names']).unstack()


In [ ]:
data.set_index(['date', 'Names']).unstack()


In [ ]:
one_hot = pd.get_dummies(dataset['Names'])
dataset = dataset.drop('Names',axis = 1)
dataset = dataset.join(one_hot)


In [ ]:
dataset